In [1]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm


In [8]:
### Load in Stock Data
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
start_data = '2022-01-01'
stock_data = import_stock_data(tickers, start_data)
# Get most recent stock price
S_0 = stock_data.iloc[1, -1]
S_0

[*********************100%%**********************]  1 of 1 completed


144.41650390625

In [11]:
### Compute Log Returns
def get_log_returns(data):
    return np.log(stock_data / stock_data.shift(1))

log_returns = get_log_returns(stock_data)
log_returns.tail()

,GOOG
Date,
2024-05-06,0.004958
2024-05-07,0.018378
2024-05-08,-0.010577
2024-05-09,0.002451
2024-05-10,-0.016928


In [13]:
### Calculate Historical Volatility
def volatility(log_returns):
    # Calculate standard deviation of log_returns
    std_dev = log_returns.std()
    # Calculate historical volatility (v_0) = sqrt(252) * std dev
    v_0 = np.sqrt(252) * std_dev # 252 trading days per year

    return v_0

v_0 = volatility(log_returns)
v_0

GOOG    0.344705
dtype: float64

In [14]:
### Initialize Parameters
# Mean-reversion speed in the Heston model
k = 0.1  
# Volatility of volatility in the Heston model
sigma = 0.2  
# Time step, typically representing trading days in a year
dt = 252  
# Long-term average variance (squared volatility) in the Heston model
theta = v_0 ** 2  
# Drift rate, computed as mean - 0.5 * variance
mean = log_returns.mean()  
var = log_returns.var()  
drift = mean - (0.5 * var)  


In [15]:
### Heston Model Function
def heston_model(S_0, v_0, drift, k, theta, sigma, dt, N, trials):
    # Generate random numbers z_1​ and z_2​ from a standard normal distribution for each time step
    z_1 = np.random.randn(N)
    z_2 = np.random.randn(N)
    # Generate array for each pricing path
    paths = []
    # Create empty lists to store asset prices
    S = np.zeros(N)
    v = np.zeros(N)
    # Set first value of array to initial S_0 and v_0 
    S[0] = S_0
    v[0] = v_0

    for t in range(1, N):
        # For each time step t, update the asset price S(t) using the asset price SDE in the Heston model algorithm
        S[t] = S[t-1]*np.exp((drift - 1/2*(v[t-1])*dt) + np.sqrt(v[t-1]*dt)*z_1)
        # Update the volatility v(t) using the volatility SDE in the Heston model algorithm
        v[t] = v[t-1] + k*(theta - v[t])*dt + sigma*np.sqrt(v[t-1]*dt)*z_2

    return S, v



In [ ]:
### Generate Price Paths
def price_paths(S_0, v_0, drift, k, theta, sigma, dt, N):
    

    for path in 